In [ ]:
import math


Gsc = Solar constant 

φ = Latitude 

L = Longitude 

δ = Declination -> **-23.45°≤δ≤23.45°** 

ω = Hour angle -> **-180°≤ω≤180°** 

hs = Sun altitude 

γ = Solar azimuth 

θz = Zenith Angle -> **0° ≤ θz ≤ 90°** 

Gon = Exact irradiation incident on a surface θ outside the atmosphere 

τb = Transmision Beam radiation 

τd = Transmision diffuse radiation 

Gcb = Beam radiation 



Solar constant

![title](http://i.imgur.com/UqAn4ZB.png)


σ = 5.67*10^-8 W/m^2.k^4 Stefan Boltzmann constant

R = 696*10^6m Sun radiuses

D = 150 * 10^9 is the avg distance between sun and earth

In [ ]:
solar_constant = 1367  # W/m^2


Sun position 

=> α=90-Φ+δ

In [7]:
def sun_position(latitude, declination):
    return 90 - latitude + declination


Where declination is calculated as

=> δ= 23.45° ⋅ Sin[ 360/365 (284+d)] 

and 284 is an adjustation value as declination not being zero on January 

In [8]:
def declination(day):
    return 23.45 * math.sin((360 / 365) * (284 + day))


The hour angle is the derivation of the sun's angle from south:

=> ω = 15° ⋅ (SolarTime - 12)

In [9]:
def hour_angle(solar_time):
    return 15 * (solar_time - 12)


Where solar time is calculated as:

=> E = 229.2 ⋅ (0.000075 + 0.001868 cosB − 0.032077 ⋅ sinB−0.014615 ⋅  cos2B − 0.04089 ⋅ sin2B)

=> B = (n - 1) ⋅ 360/365

=> SolarTime = StandardTime + 4(Lst - Lloc) + E

Lst = Standard meridian for the local timezone
Lloc = Longitude of the location

Reference: https://en.wikipedia.org/wiki/Equation_of_time

In [1]:
def e(day):
    B = (day - 1) * 360 / 365
    return 229.2 * (0.000075 + 0.001868 * math.cos(B) * math.sin(B) - 0.014615
                    * math.cos(2 * B) - 0.04089 * math.sin(2 * B))


def solar_time(std_time, std_meridian, longitude, day):
    return std_time + ((4 * (std_meridian - longitude) + e(day)) / 60)


Solar altitude:

=> hs = Arcsin[cos(φ) * cos(δ) * cos(ω) + sin(φ)*sin(δ)]

In [ ]:
def sun_altitude(latitude, declination, hour_angle):
    return math.asin(math.cos(latitude) * math.cos(declination) * math.cos(hour_angle) \
                     + math.sin(latitude) * math.sin(declination))


Solar azimuth:

=> γ = Arcsin[(cos(δ) ⋅ sin(ω))/cos(hs)]

In [ ]:
def azimuth(declination, hour_angle, sun_altitude):
    return math.asin((math.cos(declination) * math.sin(hour_angle)) / math.cos(sun_altitude))


Zenith angle:

=> cosθz = cosφ⋅cosδ⋅cosω+sinφ⋅sinδ

In [ ]:
def zenith_angle(latitude, declination, hour_angle):
    return math.cos(latitude) * math.cos(declination) * math.cos(hour_angle) + math.sin(latitude) * math.sin(
        declination)


Radiation outside atmosphere:

=> Gon = Gsc * (1 + 0.033 ⋅ cos((360⋅n)/365))

In [ ]:
def radiation_outside_atmosphere(day):
    return solar_constant * (1 + 0.033 * math.cos((360 * day) / 365))


Thau beam:

=> τb = Gcb / Go = a0 + a1 ⋅ e^(-K/cosθz)

Beam radiation:

=> Gcb = τb ⋅ Gon ⋅ (cosφ ⋅ cosδ ⋅ cosω + sinφ ⋅ sinδ)

Let's define some constants:

a0 = 0,4237 - 0.00821 ⋅ (6-A)^2
a1 = 0,5055 - 0.00595 ⋅ (6.5-A)^2
k = 0.2711 + 0.01858 ⋅ (2.5-A)^2

Where A is the altitude above the sea level (in km).

In [ ]:
def transmission_beam(altitude, zenith_angle):
    a0 = 0.4237 - 0.00821 * (6 - altitude) ** 2
    a1 = 0.5055 - 0.00595 * (6.5 - altitude) ** 2
    k = 0.2711 + 0.01858 * (2.5 - altitude) ** 2

    return a0 + a1 * (math.e ** (-k / math.cos(zenith_angle)))


def beam_radiation(t_beam, rad_outside_atmosphere, latitude, declination, hour_angle):
    return t_beam * rad_outside_atmosphere * (
        math.cos(latitude) * math.cos(declination) * math.cos(hour_angle) + math.sin(latitude) * math.sin(
            declination))


Transimion diffuse radiation:

=> τd = 0.271 - 0.294 ⋅ τb

Diffuse Radiation:

=> Gcd = τd ⋅ Gon ⋅ (cosφ ⋅ cosδ ⋅ cosω + sinφ ⋅ sinδ)

In [2]:
def transmission_diffuse_radiation(transmission_beam):
    return 0.271 - 0.294 * transmission_beam


def diffuse_radiation(t_diffuse_radiation, rad_outside_atmosphere, latitude, declination, hour_angle):
    return t_diffuse_radiation * rad_outside_atmosphere * (math.cos(latitude) * math.cos(declination)
                                                           * math.cos(hour_angle) + math.sin(latitude)
                                                           * math.sin(declination))


Total radiation received on horizontal at ground surface at a clear day:

=> Gc = (τb +τd)⋅Gsc⋅(1+0.033⋅cos(360⋅n)/365)⋅(cosφ⋅cosδ⋅cosω+sinφ⋅sinδ)

In [5]:
def total_radiation_day(transmission_beam, transmission_diffuse_radiation, latitude, declination, hour_angle, day):
    transmissions_sum = transmission_beam + transmission_diffuse_radiation

    return transmissions_sum * solar_constant * (
        1 + 0.033 * math.cos((360 * day) / 365) * (math.cos(latitude) * math.cos(declination)
                                                   * math.cos(hour_angle) + math.sin(latitude)
                                                   * math.sin(declination)))


Energy received taking into acount the inclinationg of the surface 

=> G (inclined) = G(horizontal) * Sin (α+β) / Sinα

β is the ° of the surface

In [6]:
def radiation_with_angle_simple(raw_radiation, sun_position, panel_angle):
    return raw_radiation * math.sin(sun_position + panel_angle) / math.sin(sun_position)